In [7]:
import glob
import os
import re
import mne
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mne_bids import BIDSPath, read_raw_bids


In [5]:
epoch_paths = glob.glob('../data/bids/derivatives/preprocessing/*/*res-hi*fif.gz')

for path in epoch_paths:`
    events = mne.read_events(path)[:,2]
    print(np.unique(events))

/tmp/ipykernel_19753/3040300095.py:4: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-7/sub-7_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(path)[:,2]


[10001 10002 10003 10004 10005]


/tmp/ipykernel_19753/3040300095.py:4: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-28/sub-28_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(path)[:,2]


[10001 10002 10003 10004 10005]


/tmp/ipykernel_19753/3040300095.py:4: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-34/sub-34_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(path)[:,2]


...

[10001 10002 10003 10004 10005]


/tmp/ipykernel_19753/3040300095.py:4: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-27/sub-27_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(path)[:,2]


[10001 10002 10003 10004 10005]


/tmp/ipykernel_19753/3040300095.py:4: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-8/sub-8_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(path)[:,2]


KeyboardInterrupt: 

In [11]:
bids_path = BIDSPath(root = '../data/bids',
                    subject = '40',
                    task = 'pitch',
                    run = '1',
                    datatype = 'eeg',
                    )
print(bids_path)
raw = read_raw_bids(bids_path, verbose = False)
raw = raw.pick_types(eeg = True)
events, events_ids = mne.events_from_annotations(raw)
events_ids

../data/bids/sub-40/eeg/sub-40_task-TEST_run-1


FileNotFoundError: File does not exist: ../data/bids/sub-40/eeg/sub-40_task-TEST_run-1_eeg

In [10]:
bids_path

BIDSPath(
root: ../data/bids
datatype: eeg
basename: sub-40_task-pitch_run-1)

In [144]:
np.random.seed(123)

sources = range(100)
outs = [a for a in range(100)]
np.random.shuffle(outs)
mapping = {sources[a]:outs[a] for a in(range(len(sources)))}
print(mapping)

np.random.seed(123)
input_array = np.random.randint(0,100, (1000,1000))
print(input_array)

# k = np.array(list(mapping.keys()))
v = np.array(list(mapping.values()))
print(v)

# mapping_ar = np.zeros(k.max()+1,dtype=v.dtype) #k,v from approach #1
# mapping_ar[k] = v
out = v[input_array]
print(out)


{0: 8, 1: 70, 2: 82, 3: 28, 4: 63, 5: 0, 6: 5, 7: 50, 8: 81, 9: 4, 10: 23, 11: 65, 12: 76, 13: 60, 14: 24, 15: 42, 16: 77, 17: 38, 18: 56, 19: 75, 20: 35, 21: 88, 22: 19, 23: 29, 24: 31, 25: 91, 26: 85, 27: 9, 28: 53, 29: 41, 30: 21, 31: 90, 32: 33, 33: 69, 34: 84, 35: 87, 36: 95, 37: 97, 38: 62, 39: 12, 40: 20, 41: 89, 42: 37, 43: 59, 44: 1, 45: 74, 46: 11, 47: 79, 48: 14, 49: 54, 50: 16, 51: 26, 52: 51, 53: 45, 54: 6, 55: 98, 56: 30, 57: 44, 58: 72, 59: 43, 60: 15, 61: 94, 62: 18, 63: 13, 64: 22, 65: 10, 66: 58, 67: 71, 68: 64, 69: 3, 70: 40, 71: 34, 72: 27, 73: 52, 74: 7, 75: 48, 76: 61, 77: 93, 78: 99, 79: 39, 80: 2, 81: 67, 82: 55, 83: 49, 84: 68, 85: 80, 86: 36, 87: 78, 88: 96, 89: 25, 90: 46, 91: 32, 92: 73, 93: 47, 94: 86, 95: 57, 96: 83, 97: 17, 98: 92, 99: 66}
[[66 92 98 ... 37 79  8]
 [31  0  8 ... 29 66 77]
 [50  4 42 ... 97  9 36]
 ...
 [26 41 89 ... 49 68 98]
 [32 66 89 ... 22 49 58]
 [72 13  1 ... 54 76 43]]
[ 8 70 82 28 63  0  5 50 81  4 23 65 76 60 24 42 77 38 56 75 35

In [ ]:
raw

In [192]:
raw_events, raw_events_ids = mne.events_from_annotations(raw)


Used Annotations descriptions: ['100', '150', '200', '250', '50']


In [193]:
raw_events_ids

{'100': 10001, '150': 10002, '200': 10003, '250': 10004, '50': 10005}

In [ ]:
# check event ids of bids data

In [186]:
epochs = mne.read_epochs(epoch_path, preload = False)
events = epochs.events

Reading /project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-35/sub-35_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Reading /project2/hcn1/pitch_tracking/scripts/../data/bids/derivatives/preprocessing/sub-35/sub-35_task-pitch_run-1_res-hi_desc-clean_epo.fif-1.gz ...
    Found the data of interest:
        t =    -200.00 ...     250.00 ms
        0 CTF compensation matrices available
Not setting metadata
4676 matching events found
No baseline correction applied
0 projection items activated


In [190]:
mapping = {10001: 100, 10002: 150, 10003: 200, 10004: 250, 10005: 50}
mne.EpochsArray(epochs, 
               epochs.info, 
               event_id = event_id)

    Rejecting  epoch based on EEG : ['TP10']
    Rejecting  epoch based on EEG : ['TP10']


ValueError: No matching events found for 100 (event id 10001)

In [66]:
def get_sub_and_run(fpath):
    sub = re.findall(r'sub-(\d{1,2})', npy_path)[0]
    run = re.findall(r'run-(\d{1})', npy_path)[0]
    return(sub, run)

def recode_events(events):
    mapping = {10001: 100, 10002: 150, 10003: 200, 10004: 250, 10005: 50}
    recoded_events = np.vectorize(mapping.get)(events)
    return(recoded_events)

# Get coordinates of all channels 
epoch_paths = glob.glob('../data/bids/derivatives/preprocessing/*/*res-hi*fif.gz')
pattern = re.compile('sub-(\d{1,2})_')
epoch_sub_order = list(map(pattern.findall, epoch_paths))
epoch_sub_order = list(itertools.chain(*epoch_sub_order))
pattern = re.compile('run-(\d{1,2})_')
epoch_run_order = list(map(pattern.findall, epoch_paths))
epoch_run_order = list(itertools.chain(*epoch_run_order))

npy_paths = glob.glob('../data/bids/derivatives/decoding/*/*stft_power*')
power = np.empty([])

# for each captrak file
for i, npy_path in enumerate(npy_paths):
    
    # get power
    print(f"Reading power values from: {npy_path}")
    sub_power = np.load(fpath)
    
    # get subject and run
    sub, run = get_sub_and_run(fpath)
    
    # find the right epochs file
    run_indices = [ind for ind, ele in enumerate(epoch_run_order) if ele == run]
    sub_indices = [ind for ind, ele in enumerate(epoch_sub_order) if ele == sub]
    ind = list(set(run_indices) & set(sub_indices))[0]
    epoch_path = epoch_paths[ind]
    epoch_sub, epoch_run = get_sub_and_run(epoch_path)
    if (epoch_sub != sub) & (epoch_run != run):
        sys.exit("Epoch file does not match stft_power file")
    
    # read events
    print(f"Reading events from epoch file: {epoch_path}")
    events = mne.read_events(epoch_path)
    events = recode_events(events)

    # add events to power array
    sub_power = [sub_power, events]
    print(np.shape(sub_power))
#     # get captrak file
#     dig = mne.channels.read_dig_captrak(fname = captrak_path)
    
#     # load epochs object for same subject
#     ind = epoch_sub_order.index('43')
#     epoch_fpath = epoch_paths[ind]
#     print(f"Reading info from epoch file: {epoch_fpath}")
#     info = mne.io.read_info(epoch_fpath)

#     # get coordinates
#     ch_names = info.ch_names
#     if len(ch_names) != N_CHANS:
#         print(f"ValueError: Incorrect number of channels, skipping subject {sub}!")
#         continue
#     try:
#         pos = np.stack([dig.get_positions()['ch_pos'][ch] for ch in ch_names])
#     except:
#         print(f"KeyError: {captrak_path} is incomplete, likely missing FCz, skipping subject {sub}!")
#         continue
        
#     # Add to list
#     pos = pos[..., np.newaxis]
#     if i == 0:
#         poss = pos
#     else:
#         poss = np.append(poss, pos, axis = 2)
#     print(np.shape(poss))

Reading power values from: ../data/bids/derivatives/decoding/sub-7/sub-7_task-pitch_run-1_desc-stft_power.npy
Reading events from epoch file: ../data/bids/derivatives/preprocessing/sub-7/sub-7_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz


/tmp/ipykernel_13242/3623981500.py:39: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-7/sub-7_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(epoch_path)
/home/letitiayhho/.conda/envs/pitch-tracking/lib/python3.10/site-packages/numpy/core/fromnumeric.py:2007: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


Reading power values from: ../data/bids/derivatives/decoding/sub-28/sub-28_task-pitch_run-1_desc-stft_power.npy


/tmp/ipykernel_13242/3623981500.py:39: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-28/sub-28_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(epoch_path)


Reading events from epoch file: ../data/bids/derivatives/preprocessing/sub-28/sub-28_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz
Reading power values from: ../data/bids/derivatives/decoding/sub-34/sub-34_task-pitch_run-1_desc-stft_power.npy
Reading events from epoch file: ../data/bids/derivatives/preprocessing/sub-34/sub-34_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz


...

/tmp/ipykernel_13242/3623981500.py:39: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-29/sub-29_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(epoch_path)


Reading power values from: ../data/bids/derivatives/decoding/sub-30/sub-30_task-pitch_run-1_desc-stft_power.npy
Reading events from epoch file: ../data/bids/derivatives/preprocessing/sub-30/sub-30_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz


/tmp/ipykernel_13242/3623981500.py:39: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-30/sub-30_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(epoch_path)


Reading power values from: ../data/bids/derivatives/decoding/sub-35/sub-35_task-pitch_run-1_desc-stft_power.npy
Reading events from epoch file: ../data/bids/derivatives/preprocessing/sub-35/sub-35_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz


/tmp/ipykernel_13242/3623981500.py:39: RuntimeWarning: This filename (../data/bids/derivatives/preprocessing/sub-35/sub-35_task-pitch_run-1_res-hi_desc-clean_epo.fif.gz) does not conform to MNE naming conventions. All events files should end with .eve, -eve.fif, -eve.fif.gz, -eve.lst, -eve.txt, _eve.fif, _eve.fif.gz, _eve.lst, _eve.txt or -annot.fif
  events = mne.read_events(epoch_path)


In [142]:
mapping

{0: 8,
 1: 70,
 2: 82,
 3: 28,
 4: 63,
 5: 0,
 6: 5,
 7: 50,
 8: 81,
 9: 4,
 10: 23,
 11: 65,
 12: 76,
 13: 60,
 14: 24,
 15: 42,
 16: 77,
 17: 38,
 18: 56,
 19: 75,
 20: 35,
 21: 88,
 22: 19,
 23: 29,
 24: 31,
 25: 91,
 26: 85,
 27: 9,
 28: 53,
 29: 41,
 30: 21,
 31: 90,
 32: 33,
 33: 69,
 34: 84,
 35: 87,
 36: 95,
 37: 97,
 38: 62,
 39: 12,
 40: 20,
 41: 89,
 42: 37,
 43: 59,
 44: 1,
 45: 74,
 46: 11,
 47: 79,
 48: 14,
 49: 54,
 50: 16,
 51: 26,
 52: 51,
 53: 45,
 54: 6,
 55: 98,
 56: 30,
 57: 44,
 58: 72,
 59: 43,
 60: 15,
 61: 94,
 62: 18,
 63: 13,
 64: 22,
 65: 10,
 66: 58,
 67: 71,
 68: 64,
 69: 3,
 70: 40,
 71: 34,
 72: 27,
 73: 52,
 74: 7,
 75: 48,
 76: 61,
 77: 93,
 78: 99,
 79: 39,
 80: 2,
 81: 67,
 82: 55,
 83: 49,
 84: 68,
 85: 80,
 86: 36,
 87: 78,
 88: 96,
 89: 25,
 90: 46,
 91: 32,
 92: 73,
 93: 47,
 94: 86,
 95: 57,
 96: 83,
 97: 17,
 98: 92,
 99: 66}

In [140]:
v

array([ 8, 70, 82, 28, 63,  0,  5, 50, 81,  4, 23, 65, 76, 60, 24, 42, 77,
       38, 56, 75, 35, 88, 19, 29, 31, 91, 85,  9, 53, 41, 21, 90, 33, 69,
       84, 87, 95, 97, 62, 12, 20, 89, 37, 59,  1, 74, 11, 79, 14, 54, 16,
       26, 51, 45,  6, 98, 30, 44, 72, 43, 15, 94, 18, 13, 22, 10, 58, 71,
       64,  3, 40, 34, 27, 52,  7, 48, 61, 93, 99, 39,  2, 67, 55, 49, 68,
       80, 36, 78, 96, 25, 46, 32, 73, 47, 86, 57, 83, 17, 92, 66])

In [120]:
v

array([ 8, 70, 82, 28, 63,  0,  5, 50, 81,  4, 23, 65, 76, 60, 24, 42, 77,
       38, 56, 75, 35, 88, 19, 29, 31, 91, 85,  9, 53, 41, 21, 90, 33, 69,
       84, 87, 95, 97, 62, 12, 20, 89, 37, 59,  1, 74, 11, 79, 14, 54, 16,
       26, 51, 45,  6, 98, 30, 44, 72, 43, 15, 94, 18, 13, 22, 10, 58, 71,
       64,  3, 40, 34, 27, 52,  7, 48, 61, 93, 99, 39,  2, 67, 55, 49, 68,
       80, 36, 78, 96, 25, 46, 32, 73, 47, 86, 57, 83, 17, 92, 66])

In [128]:
input_array

array([[66, 92, 98, ..., 37, 79,  8],
       [31,  0,  8, ..., 29, 66, 77],
       [50,  4, 42, ..., 97,  9, 36],
       ...,
       [26, 41, 89, ..., 49, 68, 98],
       [32, 66, 89, ..., 22, 49, 58],
       [72, 13,  1, ..., 54, 76, 43]])

In [127]:
out2

array([[58, 73, 92, ..., 97, 39, 81],
       [90,  8, 81, ..., 41, 58, 93],
       [16, 63, 37, ..., 17,  4, 95],
       ...,
       [85, 89, 25, ..., 54, 64, 92],
       [33, 58, 25, ..., 19, 54, 72],
       [27, 60, 70, ...,  6, 61, 59]])

In [102]:
mapping.values()

dict_values([10001, 10002, 10003, 10004, 10005])

In [13]:
# Read stft_power files and append all to data frame
first_sub = True
power = np.empty([])

for fpath in glob.glob("../data/bids/derivatives/decoding/*/*stft_power*"):
    print(fpath)
    sub_power = np.load(fpath)

    # Average over all channels
    sub_power = np.mean(sub_power, axis = 1)
    
    if first_sub == True:
        power = sub_power
        first_sub = False
    else:
        try: 
            power = np.append(power, sub_power, axis = 0)
        except:
            print("ValueError: stft coefficients have the wrong shape, skipping subject")
            continue
            
            
#     try: 
    print(np.shape(sub_power)) # n_epochs, n_chans, n_freqs, n_windows
    print(np.shape(power))
#     power = np.stack([power, sub_power]) # CHANGE STACKING
#     except:
#         print("ValueError: subject is probably missing a channel, skipping file")
#         continue

../data/bids/derivatives/decoding/sub-7/sub-7_task-pitch_run-1_desc-stft_power.npy
(3841, 5, 19)
(3841, 5, 19)
../data/bids/derivatives/decoding/sub-28/sub-28_task-pitch_run-1_desc-stft_power.npy
(4768, 5, 19)
(8609, 5, 19)
../data/bids/derivatives/decoding/sub-34/sub-34_task-pitch_run-1_desc-stft_power.npy
(1921, 5, 19)
(10530, 5, 19)
../data/bids/derivatives/decoding/sub-39/sub-39_task-pitch_run-1_desc-stft_power.npy
(4441, 5, 19)
(14971, 5, 19)
../data/bids/derivatives/decoding/sub-40/sub-40_task-pitch_run-1_desc-stft_power.npy
(3841, 5, 19)
(18812, 5, 19)
../data/bids/derivatives/decoding/sub-27/sub-27_task-pitch_run-1_desc-stft_power.npy
(4723, 5, 19)
(23535, 5, 19)
../data/bids/derivatives/decoding/sub-8/sub-8_task-pitch_run-1_desc-stft_power.npy
(3841, 5, 19)
(27376, 5, 19)
../data/bids/derivatives/decoding/sub-33/sub-33_task-pitch_run-1_desc-stft_power.npy
(4782, 5, 19)
(32158, 5, 19)
../data/bids/derivatives/decoding/sub-38/sub-38_task-pitch_run-1_desc-stft_power.npy
(2881, 5,

In [14]:
np.shape(power)

(152446, 5, 19)

In [10]:
np.shape(sub_power)

(4553, 19)

In [11]:
os.getcwd()

'/project2/hcn1/pitch_tracking/scripts'